## Assignment 1

### Step 1: Import Gurobi

In [1]:
import gurobipy as gp
from gurobipy import GRB
import random
import csv
import pprint

In [2]:
warehouses = {
    0:  "thunder bay",
    1:  "sudbury",
    2:  "ottawa",
    3:  "toronto",
    4:  "kingston",
    5:  "windsor",
    6:  "niagara falls",
    7:  "london"
}

retailers = {
    0:  "vaughan",
    1:  "brampton",
    2:  "toronto",
    3:  "mississauga",
    4:  "kitchener",
    5:  "hamilton",
    6:  "leamington",
    7:  "chatham kent",
    8:  "sarnia",
    9:  "stratford",
    10: "bellville",
    11: "peterborough",
    12: "orillia",
    13: "collingwood",
    14: "oshawa",
    15: "milton",
    16: "brantford",
    17: "st catherines",
    18: "welland",
    19: "woodstock"
}

In [3]:
# # Debug Cell
# print(distances)
# for m in range(len(distances)):
#     for n in range(len(distances[0])):
#         print(f"{warehouses[m]} -> {retailers[n]}: {distances[m][n]} km" )

---

## a) Base Mathematical Model

### Define Model

In [4]:
m = gp.Model()

Using license file /Users/matthewfollegot/gurobi.lic
Academic license - for non-commercial use only


### Sets

In [5]:
P = ['Barrie', 'Cambridge']

W = [w for w in warehouses.values()]

R = [r for r in retailers.values()]

### Parameters

In [13]:
# Plant capacity
Q_p = [1500, 1500]

# Warehouse Capacity
Q_w = [200, 400, 200, 800, 800, 400, 300, 600]

# Warehouse Cost
C_w = [90, 110, 150, 150, 90, 110, 110, 150]

# Transportation Cost
t = 0.187

# Retailer Demand
random.seed(20726975) # Matt's student ID
d_r = [random.randint(10000,75000) for r in range(len(R))]

# Distances
distances = []

with open ('distances.csv', newline='') as f:
    reader = csv.reader(f, delimiter=' ')
    for row in reader:
        row = [float(distance) for distance in ', '.join(row).split(',')]
        distances.append(row)

D = [
        [1441, 458, 483, 99.6, 343, 278, 129, 102],
        [1290, 308, 460, 103, 332, 425, 197, 250]
]

### Decision Variables

In [20]:
# Number of products transported from plant p to warehouse w
x1 = {}
for p in range(len(P)):
    for w in range(len(W)):
        x1[p,w] = m.addVar(lb=0.0, obj=D[p][w]*t, vtype=GRB.INTEGER, name="x1_pw_"+str(p)+str(w))

# Number of products transported from warehouse w to retailer r
x2 = {}
for w in range(len(W)):
    for r in range(len(R)):
        print(distances[w][r]*t)
        
        x2[w,r] = m.addVar(lb=0.0, obj=distances[w][r]*t, vtype=GRB.INTEGER, name="x2_wr_"+str(w)+str(r))

# Binary variable for whether or not warehouse w is open
y = {}
for w in range(len(W)):
    y[w] = m.addVar(obj=C_w[w]*1000, vtype=GRB.BINARY, name="y_"+str(w))

252.45
255.255
258.434
258.621
270.776
267.784
246.466
250.58
237.864
264.044
289.663
259.743
236.929
245.344
264.979
262.735
273.955
274.516
279.378
265.727
68.629
71.621
74.613
74.8
87.142
83.963
132.396
122.859
122.111
94.809
105.842
76.109
53.108
61.523
81.345
79.101
90.134
90.695
95.744
94.809
74.987
76.296
82.28
88.451
91.817
97.614
137.071
127.534
126.599
99.484
50.677
50.864
70.499
85.833
74.052
83.776
103.785
104.346
109.208
99.484
8.3028
6.3206
2.7302
5.1612
20.009
13.5014
64.702
55.352
54.417
27.676
35.53
25.619
27.302
30.107
11.743599999999999
11.986699999999999
19.635
20.196
25.058
27.115
51.051
51.238
46.376
52.734
65.637
61.897
110.891
101.354
100.606
73.304
14.9226
32.912
57.409
72.93
38.148
57.596
68.068
68.442
73.491
73.304
68.629
65.637
70.499
65.076
53.669
57.035
9.7988
15.090900000000001
29.172
48.994
101.728
91.256
87.516
79.101
77.979
59.653
50.677
66.759
71.80799999999999
43.384
25.993
23.562
25.806
19.635
25.619
16.4934
65.824
56.474
55.539
33.286
58.344
49.368

### Objective Function

In [21]:
m.modelSense = GRB.MINIMIZE

### Constraints

In [22]:
# Retailer Demand Constraint
for r in range(len(R)):
    tot_demand = 0
    for w in range(len(W)):
        tot_demand += x2[w,r]
    m.addConstr(tot_demand, GRB.EQUAL, d_r[r])

# Plant Production Capacity Constraint
for p in range(len(P)):
    cap_prod = 0
    for w in range(len(W)):
        cap_prod += x1[p,w]
    m.addConstr(cap_prod, GRB.LESS_EQUAL, Q_p[p]*1000)
    
# Warehouse Incoming Capacity Constraint
for w in range(len(W)):
    cap_in = 0
    for p in range(len(P)):
        cap_in += x1[p,w]
    m.addConstr(cap_in, GRB.LESS_EQUAL, Q_w[w]*1000*y[w])

# Warehouse Flow Constraint
for w in range(len(W)):
    flow_in = 0
    flow_out = 0
    for p in range(len(P)):
        flow_in += x1[p,w]
    for r in range(len(R)):
        flow_out += x2[w,r]
    m.addConstr(flow_in, GRB.EQUAL, flow_out)

m.update()

### Solve the Model

In [23]:
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 96 rows, 368 columns and 912 nonzeros
Model fingerprint: 0xc25c7294
Variable types: 0 continuous, 368 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+05]
  Objective range  [2e-01, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+04, 2e+06]

MIP start from previous solve produced solution with objective 4.61514e+07 (0.02s)
Loaded MIP start from previous solve with objective 4.61514e+07

Presolve removed 58 rows and 184 columns
Presolve time: 0.01s
Presolved: 38 rows, 184 columns, 376 nonzeros
Variable types: 0 continuous, 184 integer (8 binary)

Root relaxation: objective 4.593903e+07, 32 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      4.6151e+07 4.6151e+07  0.00%     -    0s

Explored 0 nodes (34 simplex iter

In [24]:
# print(y)
print(x1)
print(x2)
print(m.getVars())

{(0, 0): <gurobi.Var x1_pw_00 (value -0.0)>, (0, 1): <gurobi.Var x1_pw_01 (value -0.0)>, (0, 2): <gurobi.Var x1_pw_02 (value -0.0)>, (0, 3): <gurobi.Var x1_pw_03 (value 638434.0)>, (0, 4): <gurobi.Var x1_pw_04 (value -0.0)>, (0, 5): <gurobi.Var x1_pw_05 (value -0.0)>, (0, 6): <gurobi.Var x1_pw_06 (value 42472.0)>, (0, 7): <gurobi.Var x1_pw_07 (value 249296.0)>, (1, 0): <gurobi.Var x1_pw_10 (value -0.0)>, (1, 1): <gurobi.Var x1_pw_11 (value -0.0)>, (1, 2): <gurobi.Var x1_pw_12 (value -0.0)>, (1, 3): <gurobi.Var x1_pw_13 (value -0.0)>, (1, 4): <gurobi.Var x1_pw_14 (value -0.0)>, (1, 5): <gurobi.Var x1_pw_15 (value -0.0)>, (1, 6): <gurobi.Var x1_pw_16 (value -0.0)>, (1, 7): <gurobi.Var x1_pw_17 (value -0.0)>}
{(0, 0): <gurobi.Var x2_wr_00 (value -0.0)>, (0, 1): <gurobi.Var x2_wr_01 (value -0.0)>, (0, 2): <gurobi.Var x2_wr_02 (value -0.0)>, (0, 3): <gurobi.Var x2_wr_03 (value -0.0)>, (0, 4): <gurobi.Var x2_wr_04 (value -0.0)>, (0, 5): <gurobi.Var x2_wr_05 (value -0.0)>, (0, 6): <gurobi.Var

In [25]:
for myVars in m.getVars():
    #print(myVars)
    print('%s %g' % (myVars.varName, myVars.x))

x1_pw_00 0
x1_pw_01 0
x1_pw_02 0
x1_pw_03 586643
x1_pw_04 51791
x1_pw_05 56669
x1_pw_06 42472
x1_pw_07 192627
x1_pw_10 0
x1_pw_11 0
x1_pw_12 0
x1_pw_13 0
x1_pw_14 0
x1_pw_15 0
x1_pw_16 0
x1_pw_17 0
x2_wr_00 0
x2_wr_01 0
x2_wr_02 0
x2_wr_03 0
x2_wr_04 0
x2_wr_05 0
x2_wr_06 0
x2_wr_07 0
x2_wr_08 0
x2_wr_09 0
x2_wr_010 0
x2_wr_011 0
x2_wr_012 0
x2_wr_013 0
x2_wr_014 0
x2_wr_015 0
x2_wr_016 0
x2_wr_017 0
x2_wr_018 0
x2_wr_019 0
x2_wr_10 0
x2_wr_11 0
x2_wr_12 0
x2_wr_13 0
x2_wr_14 0
x2_wr_15 0
x2_wr_16 0
x2_wr_17 0
x2_wr_18 0
x2_wr_19 0
x2_wr_110 0
x2_wr_111 0
x2_wr_112 0
x2_wr_113 0
x2_wr_114 0
x2_wr_115 0
x2_wr_116 0
x2_wr_117 0
x2_wr_118 0
x2_wr_119 0
x2_wr_20 0
x2_wr_21 0
x2_wr_22 0
x2_wr_23 0
x2_wr_24 0
x2_wr_25 0
x2_wr_26 0
x2_wr_27 0
x2_wr_28 0
x2_wr_29 0
x2_wr_210 0
x2_wr_211 0
x2_wr_212 0
x2_wr_213 0
x2_wr_214 0
x2_wr_215 0
x2_wr_216 0
x2_wr_217 0
x2_wr_218 0
x2_wr_219 0
x2_wr_30 64428
x2_wr_31 62093
x2_wr_32 27374
x2_wr_33 71836
x2_wr_34 74540
x2_wr_35 51518
x2_wr_36 0
x2_wr_37 0
